[View in Colaboratory](https://colab.research.google.com/github/hjjimmykim/prospectiveporpoise/blob/master/ExactValueLearning.ipynb)

LOLA

Ideas:
1. RL vs. RL (multiagent)
2. Communication channel (pre-set? deception?)
3. LSTM stacked on FWN
4. Induce RL to learn ZD-strategy (learn probabilities?)
5. RL that adapts to opponent's non-stationary strategy

# Libraries

In [0]:
# Standard
import numpy as np
import matplotlib.pyplot as plt
import copy

import tensorflow as tf

# cuda
use_cuda = 1

# Parameters

In [0]:
#Reward structure
T = 5 # Temptation
R = 3 # Reward
P = 1 # Penalty
S = 0 # Sucker

# Reward matrix (0 = cooperate, 1 = defect)
RM = np.zeros([2,2,2])
RM[0][0][0] = R
RM[0][0][1] = R
RM[0][1][0] = S
RM[0][1][1] = T
RM[1][0][0] = T
RM[1][0][1] = S
RM[1][1][0] = P
RM[1][1][1] = P

# RL
gamma = 1
alpha = 0.01

# Tensorflow stuff
n_eps = 5200       # Number of episodes
n_turns = 10      # Number of turns per episode
n_hidden  = 5
layers = 1
epsilon = 0.5       # Exploration parameter (epsilon-greedy)
epsilon_dr = 0.9999
input_dim = 2


# 0-layer Deep Q-Network

In [5]:
# 1. Experience replay
# 2. Clamped Q-network

tf.reset_default_graph()
  
class FWN():
  
  def __init__(self, input_dim, n_hidden, output_dim):
    # x = e.g. [[0,0], [-1,1], ...]
    # h = LSTM(x) (1 x n_hidden)
    # output = Wh + b (1 x 2)
    
    self.input_dim = input_dim
    
    # Weight initialization
    self.W1 = tf.Variable(tf.random_normal([input_dim, n_hidden]))
    self.b1 = tf.Variable(tf.random_normal([n_hidden]))
    
    self.W2 = tf.Variable(tf.random_normal([n_hidden, output_dim]))
    self.b2 = tf.Variable(tf.random_normal([output_dim]))
    
  def forward(self, x):
    x = tf.matmul(x, self.W1) + self.b1
    x = tf.nn.sigmoid(x)
    
    x = tf.matmul(x, self.W2) + self.b2
    
    return x

#tf.reset_default_graph()
sess = tf.InteractiveSession() # Initialize session

# Model 1 ----------------------------------------------------------------------

# Define Q-networks
model1 = FWN(2, n_hidden, 1)

# Target network
model1_target = FWN(2, n_hidden, 1)

# Copy weights
model1_target.W1 = model1.W1 + 0
model1_target.b1 = model1.b1 + 0
model1_target.W2 = model1.W2 + 0
model1_target.b2 = model1.b2 + 0

# Feedforward operation (FWN)
x1 = tf.placeholder(shape=[None,2],dtype=tf.float32)
Qout1 = model1.forward(x1)

Qout1_target = model1_target.forward(x1)

action_history1 = tf.placeholder(shape=(),dtype=tf.int32) # Chosen action
Qout1_train = model1.forward(x1)
Qout1_train = Qout1_train[0][action_history1] # Q-estimate for the selected action

# Loss and optimizer
nextQ1 = tf.placeholder(shape=[1,1],dtype=tf.float32) # Empirical
cost1 = tf.reduce_sum(tf.square(nextQ1-Qout1_train))
optimizer1 = tf.train.GradientDescentOptimizer(learning_rate=alpha).minimize(cost1)

#cost2 = tf.reduce_sum(tf.square(nextQ2-Qout2))
#optimizer2 = tf.train.RMSPropOptimizer(learning_rate=alpha).minimize(cost2)

# Model 2 ----------------------------------------------------------------------
model2 = FWN(2, n_hidden, 5)

# Target network
model2_target = FWN(2, n_hidden, 5)

# Copy weights
model2_target.W1 = model2.W1 + 0
model2_target.b1 = model2.b1 + 0
model2_target.W2 = model2.W2 + 0
model2_target.b2 = model2.b2 + 0

# Feedforward operation (FWN)
x2 = tf.placeholder(shape=[None,2],dtype=tf.float32)
Qout2 = model2.forward(x2)

Qout2_target = model2_target.forward(x2)

action_history2 = tf.placeholder(shape=(),dtype=tf.int32) # Chosen action
Qout2_train = model2.forward(x2)
Qout2_train = Qout2_train[0][action_history2]

# Loss and optimizer
nextQ2 = tf.placeholder(shape=[1,1],dtype=tf.float32)
cost2 = tf.reduce_sum(tf.square(nextQ2-Qout2_train))
optimizer2 = tf.train.GradientDescentOptimizer(learning_rate=alpha).minimize(cost2)

sess.run(tf.global_variables_initializer()) # Initialize variables

#blah = sess.run(Qout1, feed_dict={x1:np.reshape(np.array([-1,-1,-1,-1]),[1,4])})
#print(blah)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Game

In [4]:
for i_ep in range(n_eps):
  # History of outcomes (states -1 = unknown, 0 = coop, 1 = defect)
  history = -np.ones([n_turns,2]) # Start
  
  # Keep track of calculated Q-values
  Q_pred1 = np.zeros([n_turns,2])
  Q_pred2 = np.zeros([n_turns,2])
  
  # Keep track of rewards
  rewards1 = np.zeros([n_turns,1])
  rewards2 = np.zeros([n_turns,1])
  
  for i_turn in range(n_turns):
    
    # Calculate Q-values
    state1 = history
    state2 = np.flip(state1,axis=1)
    
    # For feedforward
    state1 = np.reshape(state1,[1,2])
    state2 = np.reshape(state2,[1,2])

    # Action selection
    Q1 = sess.run(Qout1, feed_dict={x1: state1})
    
    # 0 = cooperate, 1 = defect
    if np.random.rand(1) < Q1:
      action1 = 0
    else:
      action1 = 1
      
    Q2 = sess.run(Qout2, feed_dict={x2: state2})
    
    # 0 = cooperate, 1 = defect
    if np.random.rand(1) < Q2:
      action2 = 0
    else:
      action2 = 1

    '''
    # Tit-for-tat with initial collab
    #if i_turn == 0 or history[i_turn-1][0] == 0:
    #  action2 = 0
    #else:
    #  action2 = 1
    
    # Uniform action
    #action2 = 0
    
    # Win-stay lose-shift with initial collab
    if i_turn == 0:
      action2 = 0
    elif reward2 == T or reward2 == R:
      action2 = action2
    elif reward2 == S or reward2 == P:
      action2 = int(not action2)
    '''
      
    # Get rewards
    reward1 = RM[action1][action2][0]
    reward2 = RM[action1][action2][1]
    
    rewards1[i_turn] = reward1
    rewards2[i_turn] = reward2
    
    # Next state
    history_next = copy.deepcopy(history)
    history_next[i_turn] = [action1, action2]
    state1_next = np.reshape(history_next[i_turn],[1,2])
    state2_next = np.reshape(np.flip(history_next[i_turn],axis=1),[1,2])
    
    # Q-learning (FWN)
    Q1_next = sess.run(Qout1_target, feed_dict={x1: state1_next})

    Q_target1 = reward1 + gamma * np.amax(Q1_next,axis=1,keepdims=1)
    
    Q2_next = sess.run(Qout2_target, feed_dict={x2: state2_next})

    Q_target2 = reward2 + gamma * np.amax(Q2_next,axis=1,keepdims=1)
    
    # Last turn
    if i_turn == n_turns - 1:
      Q_target1 = np.reshape(reward1,[1,1])
      Q_target2 = np.reshape(reward2,[1,1])
    
    sess.run(optimizer1, feed_dict={x1:state1, nextQ1:Q_target1, action_history1:action1})
    sess.run(optimizer2, feed_dict={x2:state2, nextQ2:Q_target2, action_history2:action2})
    
    # Update history
    history = copy.deepcopy(history_next)
  
  # Q_{s+1}
  if i_ep % 100 == 0:
    print(np.sum(rewards1))
    print(np.sum(rewards2))
    print(history)
    
    # Update target network
    # Copy weights
    model1_target.W1 = model1.W1 + 0
    model1_target.b1 = model1.b1 + 0
    model1_target.W2 = model1.W2 + 0
    model1_target.b2 = model1.b2 + 0
    
    model2_target.W1 = model2.W1 + 0
    model2_target.b1 = model2.b1 + 0
    model2_target.W2 = model2.W2 + 0
    model2_target.b2 = model2.b2 + 0

  # Epsilon-greedy schedule
  epsilon = epsilon * epsilon_dr
  if i_ep == 5000:
    print(epsilon)
    epsilon = 0
    print('------------------------------------------------')
  
#sess.close()

NameError: ignored

In [29]:
state1 = np.array([[-1,-1],[-1,-1]])
state1 = np.reshape(state1,[1,4])
print(state1)
Q1 = sess.run(Qout1, feed_dict={x1: state1})
print(Q1)

state1 = np.array([[0,0],[-1,-1]])
state1 = np.reshape(state1,[1,4])
print(state1)
Q1 = sess.run(Qout1, feed_dict={x1: state1})
print(Q1)

state1 = np.array([[1,0],[-1,-1]])
state1 = np.reshape(state1,[1,4])
print(state1)
Q1 = sess.run(Qout1, feed_dict={x1: state1})
print(Q1)

state1 = np.array([[0,1],[-1,-1]])
state1 = np.reshape(state1,[1,4])
print(state1)
Q1 = sess.run(Qout1, feed_dict={x1: state1})
print(Q1)


[[-1 -1 -1 -1]]


ValueError: ignored

In [156]:
history

array([[0., 0.],
       [1., 1.]])

In [6]:
Q_1

[[0.9433724908263161, 0.5659474045908255],
 [1.0956443850857551, 2.757813922629177],
 [0.5001720490563554, 0.6315677743814965],
 [1.7888472409760967, 9.999341552075956],
 [0, 0.1]]

In [0]:
p_1 = np.exp(Q_1[outcome][0])/np.sum(np.exp(Q_1[outcome]))

In [8]:
p_1

0.00027171249332988586

In [9]:
outcome

3

#Message Network

In [12]:
#Create a message
#First position is what you did last turn: 0 = c, 1 = d
#Next n many characters are a message from each agent
#m1 is agent 1 message and m2 is agent 2 message

n = 3
m1 = np.zeros(n+2)
m2 = np.zeros(n+2)

m1[0] = action1
m2[0] = action2
m1[1] = action2
m2[2] = action1

print(m1)

[1. 1. 0. 0. 0.]
